In [4]:
from compChem.Hartree_Fock import Molecule # package is available on github
from scipy.linalg import eigh, sqrtm
import numpy as np
import psi4

In [5]:
def basischanger(c, f):
        """
        allows a change in basis
        
        input:
        c: transformation matrix
        f: operator to be transformed
        """
        return np.einsum("ij,jm,mn->in", c.T, f, c, optimize=True)

In [54]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
        #getting the transformation matrices
        s_eigh = eigh(self.id.overlap)
        s_diag = np.diag(s_eigh[0]**(-1/2))
        self.x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
        #needed to transfrom from old to new basis, in this basis, S will equal unity, does not conserve inner product
        self.transform = np.linalg.inv(self.x)
        
        #getting the initial density matrices
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        a_trans = np.einsum("pq, qr, rs->ps", self.transform, a, self.transform.T, optimize=True)
        b_trans = np.einsum("pq, qr, rs->ps", self.transform, b, self.transform.T, optimize=True)
        self.pMatrix = (a_trans + b_trans)/2
        
    
    def setChargeDensity(self, new_matrix):
        """sets the pMatrix to a new value"""
        self.pMatrix = new_matrix
        
        
    def fock_Alternator(self):
        """
        alters a fock operator to follow CUHF, this entails transformation to a basis where S is unity and P is diagonal, also
        we will change the cv blocks
        
        """
        #get the current matrices
        f_a = self.id.displayFockMatrix("alpha")
        f_b = self.id.displayFockMatrix("beta")
        
       
        
        #we need a closed shell type operator, it is already in the correct basis
        f_cs = (f_a + f_b)/2
        
        
        #amount of paired electrons if the number of alpha electrons is greater then or equal to the number of beta electrons
        cc = self.id.beta
        
        #amount of unpaired electrons
        oo = self.id.alpha - self.id.beta
        
        #alter first blocks
        f_a[:cc + oo, cc + oo:] = f_cs[:cc + oo, cc + oo:]
        f_a[cc + oo:, :cc + oo] = f_cs[cc + oo:, :cc + oo]
        
        f_b[:cc + oo, cc + oo:] = f_cs[:cc + oo, cc + oo:]
        f_b[cc + oo:, :cc + oo] = f_cs[cc + oo:, :cc + oo]
        
        
        return f_a, f_b
        
    def iteratinator(self):
        """adapted version of the original iterator function in the compChem.Hartree_Fock Molecule object"""
        #we will set a guess using a CUHF operator
        f_a = self.id.getDensityMatrix("alpha")
        f_b = self.id.getDensityMatrix("beta")
        
        #transfrom to basis where S is unity
        f_a_trans = np.einsum("pq, qr, rs->ps", self.transform, f_a, self.transform.T, optimize=True)
        f_b_trans = np.einsum("pq, qr, rs->ps", self.transform, f_b, self.transform.T, optimize=True)
        
        #transfrom to basis where P is diagonal, does conserve inner product
        c = eigh(self.pMatrix)[1]
        f_a_no = np.einsum("pq, qr, rs->ps", c.T, f_a_trans, c, optimize=True)
        f_b_no = np.einsum("pq, qr, rs->ps", c.T, f_b_trans, c, optimize=True)
        
        self.id.setGuess(f_a_no, "alpha")
        self.id.setGuess(f_b_no, "beta")
        
        f_a, f_b = self.fock_Alternator()

        self.id.setGuess(f_a, "alpha")
        self.id.setGuess(f_b, "beta")
        
        # setting up entry parameters for the while loop
        E_new = 0  
        E_old = 0
        convergence = False

        # step 2: start iterating
        itercount = 0
        while not convergence and itercount < 1000:

            # calculating block: calculates energies
            E_new = self.id.getElectronicEnergy()
            E_total = self.id.getTotalEnergy()

            # generating block: generates new matrices UHF: account for alpha and beta
            f_a_n, f_b_n = self.fock_Alternator()
            self.id.setGuess(f_a_n, "alpha")
            self.id.setGuess(f_b_n, "beta") 
            self.updatePMatrix()

            # comparing block: will answer the "Are we there yet?" question
            if abs(E_old - E_new) < self.id.converge:
                convergence = True


            # maintenance block: keeps everything going
            print(f"iteration: {itercount}, E_tot: {E_total: .8f}, E_elek: {E_new: .8f}, deltaE: {E_new - E_old: .8f}")
            E_old = E_new

            itercount += 1
        
        return E_total
    
    
    def updatePMatrix(self):
        """updates pMatrix to current self.id densitymatrices"""
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        self.pMatrix = (a + b)/2

In [55]:
psi4.set_options({"BASIS": "cc-pvdz"})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-10)
h1.iteratinator()

iteration: 0, E_tot: -1.34936023, E_elek: -2.93689187, deltaE: -2.93689187
iteration: 1, E_tot: -1.43991504, E_elek: -3.02744667, deltaE: -0.09055480
iteration: 2, E_tot: -1.44301545, E_elek: -3.03054708, deltaE: -0.00310041
iteration: 3, E_tot: -1.44457797, E_elek: -3.03210960, deltaE: -0.00156252
iteration: 4, E_tot: -1.44419626, E_elek: -3.03172789, deltaE:  0.00038171
iteration: 5, E_tot: -1.44361003, E_elek: -3.03114166, deltaE:  0.00058623
iteration: 6, E_tot: -1.44312098, E_elek: -3.03065261, deltaE:  0.00048905
iteration: 7, E_tot: -1.44278374, E_elek: -3.03031537, deltaE:  0.00033724
iteration: 8, E_tot: -1.44256963, E_elek: -3.03010126, deltaE:  0.00021411
iteration: 9, E_tot: -1.44243985, E_elek: -3.02997149, deltaE:  0.00012977
iteration: 10, E_tot: -1.44236338, E_elek: -3.02989501, deltaE:  0.00007648
iteration: 11, E_tot: -1.44231909, E_elek: -3.02985072, deltaE:  0.00004429
iteration: 12, E_tot: -1.44229368, E_elek: -3.02982532, deltaE:  0.00002540
iteration: 13, E_tot: 

-1.4422571044514227

In [16]:
psi4.set_options({"BASIS": "cc-pvdz"})
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")


In [30]:
a = h_test.id.getDensityMatrix('alpha')
a_trans = np.einsum("pq, qr, rs->ps", np.linalg.inv(x), a, np.linalg.inv(x.T), optimize=True)
eigh(a_trans)

(array([-5.86150882e-17, -3.22652696e-17, -1.56050443e-17, -4.88077671e-18,
         2.35243178e-34,  1.69068677e-33,  7.28762949e-18,  1.01630868e-17,
         1.26454203e-17,  1.87951725e-17,  3.72325862e-17,  4.44089210e-16,
         8.88178420e-16,  1.00000000e+00,  1.00000000e+00]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.16588590e-01,  7.87285533e-01,  0.00000000e+00],
        [ 1.61538294e-02, -2.21228014e-01,  2.93838170e-01,
          3.87171109e-02,  8.34033739e-18,  4.36570598e-17,
         -2.88121935e-02, -2.23371350e-01,  2.33095702e-02,
          4.45764885e-02, -1.31583929e-01, -5.71584458e-01,
         -5.36766115e-01,  4.20386058e-01, -2.89242083e-02],
        [-1.78386115e-01, -2.09313783e-01,  4.99252094e-02,
          1.17947030e-01,  5.43353495e-18,  2.725

In [18]:


s = h_test.id.overlap
p = h_test.pMatrix
s_eigh = eigh(s)
s_diag = np.diag(s_eigh[0]**(-1/2))
x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
x.T.dot(s).dot(x)

array([[ 1.00000000e+00, -2.75821033e-15, -7.25114413e-16,
        -9.59376278e-18, -4.85722573e-17, -5.34294831e-16,
         2.98372438e-16, -2.32452946e-16,  3.08507328e-18,
         3.67761377e-16, -3.12250226e-16,  3.46944695e-18,
        -2.44596010e-16,  3.65580378e-18, -3.60822483e-16],
       [-3.01191364e-15,  1.00000000e+00, -5.44703171e-16,
         2.59107788e-16,  6.72205347e-18,  1.16053001e-15,
        -1.19088767e-15, -8.47846099e-16, -3.52038566e-17,
        -1.04083409e-17,  1.73645820e-15, -1.19522447e-15,
        -1.13277443e-15,  3.05825019e-17,  6.93889390e-17],
       [-7.56339436e-16, -5.96744876e-16,  1.00000000e+00,
        -9.72211839e-17, -9.02056208e-17, -5.55111512e-17,
        -9.02056208e-17,  3.72098186e-16,  2.55856624e-17,
        -2.22044605e-16, -9.71445147e-17, -9.64506253e-16,
         2.68882139e-16,  4.05842345e-17,  8.32667268e-17],
       [-9.59376278e-18,  2.59107788e-16, -9.72211839e-17,
         1.00000000e+00, -9.79790736e-16,  4.53916406

In [121]:
a = h_test.id.getDensityMatrix('alpha')
b = h_test.id.getDensityMatrix('beta')
x_i = np.linalg.inv(x)
x_i_t = np.linalg.inv(x.T)
a_trans = x_i.dot(a).dot(x_i_t)
b_trans = x_i.dot(b).dot(x_i_t)
p = (a_trans + b_trans)/2
c = eigh(p)
c

(array([-6.00687650e-18, -4.75866979e-18, -3.31473633e-18, -2.27268212e-18,
        -2.20868761e-18, -7.67134458e-33,  4.58363817e-34,  1.80030792e-33,
         5.22874328e-19,  1.93073916e-17,  4.00913763e-17,  6.23349833e-17,
         4.44089210e-16,  5.00000000e-01,  1.00000000e+00]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.16588591e-01,  6.31105095e-01, -4.70664284e-01],
        [ 1.28778561e-01, -1.33799819e-02, -9.08902394e-02,
          2.20571569e-01, -4.32896469e-02,  3.40471047e-15,
         -1.87520514e-16, -1.51128824e-15, -1.38953325e-01,
         -1.03370758e-01,  2.25640737e-01,  6.14706453e-01,
         -5.36766113e-01,  3.19698738e-01, -2.74506401e-01],
        [-1.09393643e-01,  3.48007069e-01, -2.88241726e-02,
          2.71966661e-02,  3.98216954e-01, -1.693

Up untill here everything should be correct (density matrices are idempotent, p matrix has correct eigenvalues). The area below is still under construction.

In [90]:
f_a = h_test.id.displayFockMatrix('alpha')
f_b = h_test.id.displayFockMatrix('beta')

In [125]:
f_a_trans = x.dot(f_a).dot(x.T)
f_a_no = c[1].T.dot(f_a_trans).dot(c[1])
f_b_trans = x.dot(f_b).dot(x.T)
f_b_no = c[1].T.dot(f_b_trans).dot(c[1])

This expression seems a bit too simple. Nevertheless, according to Tsuchimochi & Scuseria it should hold.

In [133]:
f_cs = (f_a_no + f_b_no)/2

In [134]:
f_a_no[:3, 3:] = f_cs[:3, 3:]
        
f_a_no[3:, :3] = f_cs[3:, :3]

In [136]:
f_b_no[:3, 3:] = f_cs[:3, 3:]
        
f_b_no[3:, :3] = f_cs[3:, :3]

In [137]:
h_test.id.setGuess(f_trans, "alpha")
h_test.id.setGuess(g_trans, "beta")

h_test.id.iterator()

/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":
/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"


iteration: 0, E_tot:  4.46297122, E_elek:  2.87543959, deltaE:  2.87543959, rmsD:  44.37050317
iteration: 1, E_tot: -1.40939335, E_elek: -2.99692498, deltaE: -5.87236457, rmsD:  4.48238302
iteration: 2, E_tot: -1.49442064, E_elek: -3.08195228, deltaE: -0.08502729, rmsD:  1.29705008
iteration: 3, E_tot: -1.50286682, E_elek: -3.09039846, deltaE: -0.00844618, rmsD:  0.51440015
iteration: 4, E_tot: -1.50446327, E_elek: -3.09199490, deltaE: -0.00159644, rmsD:  0.24794128
iteration: 5, E_tot: -1.50487610, E_elek: -3.09240773, deltaE: -0.00041283, rmsD:  0.12718140
iteration: 6, E_tot: -1.50499978, E_elek: -3.09253142, deltaE: -0.00012369, rmsD:  0.06644183
iteration: 7, E_tot: -1.50504065, E_elek: -3.09257228, deltaE: -0.00004087, rmsD:  0.03491188
iteration: 8, E_tot: -1.50505561, E_elek: -3.09258724, deltaE: -0.00001496, rmsD:  0.02267563
iteration: 9, E_tot: -1.50506201, E_elek: -3.09259364, deltaE: -0.00000640, rmsD:  0.02026409
iteration: 10, E_tot: -1.50506549, E_elek: -3.09259712, del

iteration: 96, E_tot: -1.50606884, E_elek: -3.09360048, deltaE: -0.00001114, rmsD:  0.06777612
iteration: 97, E_tot: -1.50607953, E_elek: -3.09361116, deltaE: -0.00001068, rmsD:  0.06624965
iteration: 98, E_tot: -1.50608976, E_elek: -3.09362139, deltaE: -0.00001023, rmsD:  0.06471913
iteration: 99, E_tot: -1.50609956, E_elek: -3.09363119, deltaE: -0.00000979, rmsD:  0.06318813
iteration: 100, E_tot: -1.50610892, E_elek: -3.09364055, deltaE: -0.00000936, rmsD:  0.06165997
iteration: 101, E_tot: -1.50611786, E_elek: -3.09364949, deltaE: -0.00000894, rmsD:  0.06013777
iteration: 102, E_tot: -1.50612638, E_elek: -3.09365801, deltaE: -0.00000853, rmsD:  0.05862439
iteration: 103, E_tot: -1.50613451, E_elek: -3.09366614, deltaE: -0.00000813, rmsD:  0.05712322
iteration: 104, E_tot: -1.50614225, E_elek: -3.09367388, deltaE: -0.00000774, rmsD:  0.05563774
iteration: 105, E_tot: -1.50614961, E_elek: -3.09368124, deltaE: -0.00000736, rmsD:  0.05416814
iteration: 106, E_tot: -1.50615661, E_elek: 

iteration: 191, E_tot: -1.50627378, E_elek: -3.09380541, deltaE: -0.00000004, rmsD:  0.00352557
iteration: 192, E_tot: -1.50627381, E_elek: -3.09380544, deltaE: -0.00000003, rmsD:  0.00341311
iteration: 193, E_tot: -1.50627384, E_elek: -3.09380548, deltaE: -0.00000003, rmsD:  0.00330425
iteration: 194, E_tot: -1.50627387, E_elek: -3.09380551, deltaE: -0.00000003, rmsD:  0.00319887
iteration: 195, E_tot: -1.50627390, E_elek: -3.09380553, deltaE: -0.00000003, rmsD:  0.00309686
iteration: 196, E_tot: -1.50627393, E_elek: -3.09380556, deltaE: -0.00000003, rmsD:  0.00299812
iteration: 197, E_tot: -1.50627395, E_elek: -3.09380558, deltaE: -0.00000002, rmsD:  0.00290253
iteration: 198, E_tot: -1.50627398, E_elek: -3.09380561, deltaE: -0.00000002, rmsD:  0.00281000
iteration: 199, E_tot: -1.50627400, E_elek: -3.09380563, deltaE: -0.00000002, rmsD:  0.00272042
iteration: 200, E_tot: -1.50627402, E_elek: -3.09380565, deltaE: -0.00000002, rmsD:  0.00263371
iteration: 201, E_tot: -1.50627404, E_el

iteration: 287, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00015821
iteration: 288, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00015319
iteration: 289, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00014832
iteration: 290, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00014361
iteration: 291, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00013904
iteration: 292, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00013463
iteration: 293, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00013035
iteration: 294, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00012621
iteration: 295, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00012220
iteration: 296, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00011831
iteration: 297, E_tot: -1.50627432, E_el

iteration: 383, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000713
iteration: 384, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000690
iteration: 385, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000668
iteration: 386, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000647
iteration: 387, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000627
iteration: 388, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000607
iteration: 389, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000587
iteration: 390, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000569
iteration: 391, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000551
iteration: 392, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000533
iteration: 393, E_tot: -1.50627432, E_el

-1.5062743202993452

In [43]:
d = np.array([[1, 3], [3, 4]])
c = eigh(d)
d_t = c[1].T.dot(d).dot(c[1])
eigh(d_t), c

((array([-0.85410197,  5.85410197]),
  array([[-1.00000000e+00, -1.32401822e-16],
         [-1.32401822e-16,  1.00000000e+00]])),
 (array([-0.85410197,  5.85410197]),
  array([[-0.85065081,  0.52573111],
         [ 0.52573111,  0.85065081]])))

In [97]:
a = h_test.id.getDensityMatrix("alpha")
a_trans = z.T.dot(a).dot(z)

a_trans

array([[ 7.86993446e-23, -4.40492022e-29, -1.60209546e-24,
         1.10167228e-16,  6.31851038e-13,  6.80554947e-13,
        -2.47474297e-16, -6.62364160e-13,  5.36309038e-14,
         2.54238109e-14,  3.77365032e-13,  7.03392293e-13,
        -5.30833893e-14,  4.65973030e-12, -8.89193826e-12],
       [-4.40492022e-29,  2.76036507e-35,  8.96723218e-31,
        -6.16626961e-23, -2.50296198e-19, -5.15076464e-19,
         1.91198853e-22,  4.01685377e-19, -6.81987441e-20,
        -1.00010858e-20, -2.88968418e-19, -5.07425163e-19,
         2.10958303e-20, -1.66079758e-18,  5.69177369e-18],
       [-1.60209546e-24,  8.96723218e-31,  3.26141200e-26,
        -2.24269232e-18, -1.28624713e-14, -1.38544592e-14,
         5.03798893e-18,  1.34839228e-14, -1.09185579e-15,
        -5.17547528e-16, -7.68225020e-15, -1.43193185e-14,
         1.08060860e-15, -9.48568423e-14,  1.81016193e-13],
       [ 1.10167228e-16, -6.16626961e-23, -2.24269232e-18,
         1.54217524e-10,  8.84480564e-07,  9.52694805

In [45]:
a_trans.dot(a_trans)

array([[ 1.31509744e-32, -1.14508632e-33,  3.39607801e-48,
        -1.61963005e-49,  2.05562799e-49, -9.73656549e-38,
         6.24182271e-35, -1.09179883e-33, -6.00573899e-34,
         8.81299316e-33, -6.04751451e-33, -1.33620208e-33,
         6.66989478e-33,  5.38281907e-17, -3.80353931e-17],
       [ 2.55329854e-33,  1.82436891e-33,  1.34535402e-47,
         1.56847134e-49,  1.23531439e-48,  4.82548374e-37,
         1.88662530e-34, -4.78299584e-34,  1.31304341e-33,
         5.06955630e-34, -8.48027587e-33,  3.23997069e-34,
         1.42303235e-33,  5.40898234e-18,  5.72211595e-17],
       [ 1.46042575e-47,  1.16065337e-47,  8.83747223e-62,
         1.07094341e-63,  8.17295333e-63,  3.26345890e-51,
         1.35252332e-48, -2.70869588e-48,  9.32448840e-48,
         2.66324006e-48, -5.41008378e-47,  2.35858984e-48,
         8.41908870e-48,  3.15960266e-32,  3.71914730e-31],
       [ 1.79514095e-50,  1.35123144e-49,  8.48917413e-64,
         1.24083986e-65,  7.91306457e-65,  3.28005275

In [32]:
c = eigh(a)[0]

In [33]:
b = h_test.id.getDensityMatrix("beta")
b.trace()

0.4718682903812628

In [34]:
d = eigh(b)[0]

In [35]:
e = np.diag(c) + np.diag(d)
e/2

array([[-1.01658530e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.29458077e-18,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.57922657e-19,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.53354521e-19,  0.00000000e+00,  0.00000000